# Convolutional Neural Network CNN

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
#Hyper-parameters
num_epochs = 100
batch_size = 4
learning_rate = 1e-3

In [4]:
# dataset has PILImage of range [0,1]
# We will transform them to Tensor of normalized range [-1,1]

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ]
)

In [6]:
train_dataset = torchvision.datasets.CIFAR10(root='data/', train=True,
                                            download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='data/', train=False,
                                            download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                          shuffle=False)

In [8]:
classes = ('plane', 'car','bird','cat',
          'deer','dog','frog','horse','ship','truck')

In [9]:
# implement conv net
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool =  nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [10]:
model = ConvNet().to(device)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [12]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4,3,32,32] = 4,3,1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad() #empty gradient first
        loss.backward()
        optimizer.step()

        if (i+1)% 2000==0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
print('Finished Training')

Epoch [1/100], Step [2000/12500], Loss: 2.2681
Epoch [1/100], Step [4000/12500], Loss: 2.3319
Epoch [1/100], Step [6000/12500], Loss: 2.3026
Epoch [1/100], Step [8000/12500], Loss: 2.3852
Epoch [1/100], Step [10000/12500], Loss: 2.1450
Epoch [1/100], Step [12000/12500], Loss: 1.9747
Epoch [2/100], Step [2000/12500], Loss: 1.7123
Epoch [2/100], Step [4000/12500], Loss: 1.7398
Epoch [2/100], Step [6000/12500], Loss: 2.0464
Epoch [2/100], Step [8000/12500], Loss: 1.6871
Epoch [2/100], Step [10000/12500], Loss: 2.1635
Epoch [2/100], Step [12000/12500], Loss: 1.5650
Epoch [3/100], Step [2000/12500], Loss: 1.6480
Epoch [3/100], Step [4000/12500], Loss: 2.0406
Epoch [3/100], Step [6000/12500], Loss: 2.6192
Epoch [3/100], Step [8000/12500], Loss: 1.5399
Epoch [3/100], Step [10000/12500], Loss: 1.0862
Epoch [3/100], Step [12000/12500], Loss: 1.4789
Epoch [4/100], Step [2000/12500], Loss: 1.3597
Epoch [4/100], Step [4000/12500], Loss: 1.3841
Epoch [4/100], Step [6000/12500], Loss: 1.3909
Epoch [

In [13]:
with torch.no_grad(): # torch.no_grad() because we dont need backward propagation
    n_correct = 0
    n_samples= 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value, index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted==labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label== pred):
                n_class_correct[label]+=1
            n_class_samples[label] +=1

    acc = 100.0 *  n_correct / n_samples
    print(f'Accuracy of the network: {acc}%')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc}%')



Accuracy of the network: 60.53%
Accuracy of plane: 67.7%
Accuracy of car: 79.0%
Accuracy of bird: 51.9%
Accuracy of cat: 30.4%
Accuracy of deer: 56.6%
Accuracy of dog: 45.5%
Accuracy of frog: 76.1%
Accuracy of horse: 66.5%
Accuracy of ship: 70.7%
Accuracy of truck: 60.9%


## Another way to represent CNN

In [ ]:
def imshow(img):
    img = img/2 +0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()

In [ ]:
dataiter = iter(train_loader)
images, labesls = next(dataiter)

In [ ]:
imshow(torchvision.utils.make_grid(images))

In [ ]:
# implement conv net
conv1 = nn.Conv2d(3,6,5)
pool = nn.MaxPool2d(2,2)
conv2 = nn.Conv2d(6,16,5)
print(images.shape)
x=conv1(images)
print(x.shape)
x=pool(x)
print(x.shape)
x=conv2(x)
print(x.shape)
x=pool(x)
print(x.shape)